# EMG Hand Gesture Classification

This notebook is used to explore the approaches from "A Robust, Real-Time Control Scheme for
Multifunction Myoelectric Control" (Englehart & Hudgins, 2003), while using the data provided by the paper "Latent Factors Limiting the Performance of sEMG-Interfaces" (Lobov et al., 2018), recorded using a MYO Thalmic bracelet.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os

### Data Preparation
This section serves to take the raw data, and reshape it/preprocess it before any real analysis being carried out.
- Data structure: dictionary of lists (keys are subject number), with lists each containing two dataframes corresponding to individual series

In [ ]:
DIR = '../' # where data directories are found relative to code
exclude = [3, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 20, 23, 27, 35] # exclude these subjects due to ADC saturation 
DIR_SJs = []
for idx in range(1, 37):
    if idx not in exclude:
        current_dir = str(idx)
        if idx < 10:
            current_dir = '0' + current_dir
        DIR_SJs.append(current_dir)

In [ ]:
def parse_file(DIR, DIR_SJ, series_n):
    ''' Given a file path, extracts numerical information based on text file provided. Returns dataframe'''
    filename = os.listdir(DIR + DIR_SJ)[series_n] # either the first or second series is extracted
    file = open(DIR + DIR_SJ + filename) # open the file to be analysed
    lines = file.readlines()
    columns = lines[0].split() # take the first row as the column labels for dataframe
    lines = lines[1:] # exclude first line which represents column labels
    data = np.zeros((len(lines), 10)) # prepare dataframe to be appropriate size
    for idx, line in enumerate(lines):
        line = line.split() # get entries into different elements
        # print(np.array(line).astype(np.float64))
        data[idx, :] = np.array(line).astype(np.float64) # convert to numerical and store in array
    file.close()
    for ch in range(1, 9): # interpolate channel values as to make data regularly sampled
        f = scipy.interpolate.interp1d(data[:, 0], data[:, ch], fill_value='extrapolate') # create interpolator based on regular data
        data[:, ch] = f(list(range(data.shape[0])))
    data[:, 0] = list(range(data.shape[0]))
    df = pd.DataFrame(data=data, columns=columns)
    df[['time', 'class']] = df[['time', 'class']].astype(np.int32) # convert timepoint and class columns to integer type
    return df

def extract_all_data(DIR, DIR_SJs):
    ''' Extracts dataframe from every single subject and series of actions. '''
    series_ns = [0,1] # possible series number
    all_data = {DIR_SJ:[] for DIR_SJ in DIR_SJs} # initialise dictionary to contain data to be analysed
    for DIR_SJ in DIR_SJs:
        for series_n in series_ns:
            df = parse_file(DIR, DIR_SJ, series_n) # get individual data file
            all_data[DIR_SJ].append(df) # add df to the list of that particular subject
    return all_data

def plot_channels(df):
    ''' Plots all channels in a given dataframe '''
    plt.figure()
    leg = []
    for idx in range(1, 9):
        plt.plot(df['time'], df['channel{}'.format(idx)])
        leg.append('channel{}'.format(idx))
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (V)')
    plt.ylabel('Single Subject Data')
    plt.legend(leg)
    plt.show()

In [ ]:
df = parse_file(DIR, '01/', 0)
df

In [ ]:
# Plot signals
for DIR_SJ in DIR_SJs:
    for series_n in range(2):
        print(DIR_SJ, series_n) # determine which signals are definitely not okay to use
        df = parse_file(DIR, DIR_SJ + '/', 0)
        plot_channels(df)

# # Plot class labels
# plt.figure()
# plt.plot(df['time'], df['class'])
# plt.xlabel('Time (ms)')
# plt.ylabel('Voltage (V)')
# plt.ylabel('Single Subject Data')
# plt.show()

# Check that sampling rate matches expectations:
print('Estimated Sampling Rate: ', df.shape[0]/df.loc[df.shape[0]-1, 'time'])

### Extracting windows from data

In [ ]:
wsize = 250 # 250ms window size, as in Ecklehart & Hudgins
